In [1]:
import pandas as pd

In [5]:
data = pd.read_csv('/content/surge_data.csv')

In [6]:
X = data.drop('flag', axis=1)
y = data['flag']

## Tomek Link

In [7]:
from imblearn.under_sampling import TomekLinks

In [8]:
tomek_links = TomekLinks()

In [9]:
X_resampled, y_resampled = tomek_links.fit_resample(X, y)

In [10]:
resampled_data = pd.DataFrame(X_resampled, columns=X.columns)
resampled_data['flag'] = y_resampled

In [11]:
print(resampled_data['flag'].value_counts())

flag
0.0    9926
1.0      69
Name: count, dtype: int64


## Logistic Regression

In [12]:
pip install river

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 9.3 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
gensim 4.3.3 requires scipy<

In [13]:
from river import compose
from river import preprocessing
from river import linear_model
from river import metrics

In [14]:
x = resampled_data[['voltage', 'current', 'power_consumption']]
y = resampled_data['flag']

In [15]:
model = compose.Pipeline(
    ('scaler', preprocessing.StandardScaler()),
    ('classifier', linear_model.LogisticRegression())
)

In [16]:
accuracy_metric = metrics.Accuracy()

In [17]:
confusion_matrix_metric = metrics.ConfusionMatrix()

In [18]:
for xi, yi in zip(x.to_dict(orient='records'), y):
    y_pred = model.predict_one(xi)
    model.learn_one(xi, yi)
    accuracy_metric.update(yi, y_pred)
    confusion_matrix_metric.update(yi, y_pred)

In [19]:
print(f'Final accuracy: {accuracy_metric}')

Final accuracy: Accuracy: 99.30%


In [20]:
print("Confusion Matrix:")
print(confusion_matrix_metric)

Confusion Matrix:
      0.0     1.0  
0.0   9,925     1  
1.0      69     0  
